<a href="https://colab.research.google.com/github/hkaido0718/SupportRestriction/blob/main/CessationLength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let $Y(z)=(Y_1(z),\dots,Y_5(z))$ be the subject's potential outcome across the five survey waves at the treatment status $z\in \{C, SIP, SIA\}$. Let $L(z)$ denote the potential length of cessation, defined as the duration of the initial spell where $Y_t(z)=0$. Specifically, $L(z)=\max\{t\in\{0,1,\dots,5\}:Y_1(z)=Y_2(z)=\cdots=Y_t(z)=0\}$.
We first consider the null hypothesis that treatment has no effect on cessation length:
\begin{align}
L(C)= L(SIP)= L(SIA).
\end{align}
We construct a potential response graph below.

In [ ]:
!git clone https://github.com/hkaido0718/SupportRestriction.git

Cloning into 'SupportRestriction'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 12), reused 5 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 725.68 KiB | 7.56 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
import networkx as nx
import itertools
from SupportRestriction.graph_analysis_utils import GraphAnalyzer

# Define nodes
binary5 = list(itertools.product([0, 1], repeat=5))
nodes = [(y, z) for z in ["C", "SIP", "SIA"] for y in binary5]

# Group function
group_fn = lambda node: node[1]  # group by z

def get_L(y):
    if y[0] != 0:
        return 0
    count = 1
    for val in y[1:]:
        if val == 0:
            count += 1
        else:
            break
    return count

def violates_equal_L(u,v):
    y1,z1 = u
    y2,z2 = v
    L1 = get_L(y1)
    L2 = get_L(y2)
    if z1 != z2 and L1 != L2:
            return True
    return False

# Build G
G = GraphAnalyzer.build_graph_pairwise(nodes, violates_equal_L,group_fn)
print(G)

Graph with 96 nodes and 1026 edges


The graph has 96 nodes with 1026 edges. We obtain its maximal independent sets below.

In [ ]:
# Instantiate analyzer
analyzer = GraphAnalyzer(G, group_fn)

# Obtain MIS
analyzer.print_mis_excluding_single_group()
df = analyzer.save_vertex_mis_incidence("incidence_matrix_L_equal.csv")

The code returns 726 maximal independent sets that can be used to test the hypothesis. The vertex-incidence matrix is saved as csv file. Finally, we check the regularity of the model. We note that the graph is high dimensional. So, to check $G$'s perfectness, it is recommended to use the polynomial-time algorithm of Chudnovsky et. al. (2020) implemented in SageMath after exporting the graph defined here. SageMath can be run on CoCalc (https://cocalc.com/). It takes less than 10 seconds to check perfectness. The following block describes the steps needed. This repository also contains another notebook `CessationLength_Sage.ipynb` that summarizes the steps handled by SageMath.

In [ ]:
nx.write_edgelist(G, "G.edgelist", data=False)
with open("G.edgelist", "w") as f:
    for u, v in G.edges():
        f.write(f"{repr((u, v))}\n")  # write the full edge tuple as a string

# After this step, place the edgelist in the current directory for SageMath.
# Execute the following (commented) codes on SageMath
# with open("G.edgelist", "r") as f:
#     edges = [tuple(eval(x.strip())) for x in f.readlines()]
# G = Graph(edges)
# G.is_perfect()

SageMath should confirm that the graph is perfect. It is easy to check the second regularity condition.

In [ ]:
for row in analyzer.enumerate_cliques_and_check(violates_equal_L):
    status = "✅" if row['satisfies_condition'] else "❌"
    print(f"{row['index']:>2}: {row['clique']} — {status}")

The model is regular. Hence, the 726 inequalities are indeed sharp.

Next, we examine a monotonicity restriction that assumes more intensive treatments weakly extend cessation length:
\begin{align}
L(C)\leqslant L(SIP)\leqslant L(SIA).
\end{align}
We define a new graph for this exercise.

In [ ]:
def violates_monotone_L(u,v):
    y1,z1 = u
    y2,z2 = v
    L1 = get_L(y1)
    L2 = get_L(y2)
    if (z1 == "C" and z2 == "SIP" and L1 > L2) or (z1 == "SIP" and z2 == "C" and L1 < L2):
            return True
    if (z1 == "C" and z2 == "SIA" and L1 > L2) or (z1 == "SIA" and z2 == "C" and L1 < L2):
            return True
    if (z1 == "SIP" and z2 == "SIA" and L1 > L2) or (z1 == "SIA" and z2 == "SIP" and L1 < L2):
            return True
    return False

Gmono = GraphAnalyzer.build_graph_pairwise(nodes, violates_monotone_L,group_fn)
print(Gmono)

analyzer = GraphAnalyzer(Gmono, group_fn)

Graph with 96 nodes and 2049 edges


The graph has 2049 edges. The following code generates MISs and save the vertex-MIS incidence matrix, which can be used for implementing the AS and RSW tests.

In [ ]:
# Obtain MIS
analyzer.print_mis_excluding_single_group()
df = analyzer.save_vertex_mis_incidence("incidence_matrix_L_order.csv")

Checking the regularity is as before. We provide the code and description below.

In [ ]:
# Output edgelist
nx.write_edgelist(Gmono, "Gmono.edgelist", data=False)
with open("Gmono.edgelist", "w") as f:
    for u, v in G.edges():
        f.write(f"{repr((u, v))}\n")


# On SageMath execute the following
# with open("Gmono.edgelist", "r") as f:
#     edges = [tuple(eval(x.strip())) for x in f.readlines()]
# Gmono = Graph(edges)
# print(Gmono)
# Gmono.is_perfect()

for row in analyzer.enumerate_cliques_and_check(violates_monotone_L):
    status = "✅" if row['satisfies_condition'] else "❌"
    print(f"{row['index']:>2}: {row['clique']} — {status}")